In [1]:
import os
import sys
import scipy
from numba import njit, prange
import numpy as np
import scipy.stats as stats
import bayesflow as bf
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
import pickle
import tensorflow as tf
import pandas as pd 
from sklearn.covariance import EmpiricalCovariance
import priors_and_simulators as ps
# Suppress scientific notation for floats
np.set_printoptions(suppress=True)
RNG = np.random.default_rng(2023)

2024-10-24 12:06:20.343880: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 12:06:20.366363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 12:06:20.696131: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/bayesflow/trainers.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook imp

In [2]:
#Settings
# Path to data
PATH = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/git/data/prepared_data/'

# Where to save files
PATH_TO_SAVE = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/git/data/pickle_gsr/'


# Load neural networks from checkpoint

In [3]:
PARAM_NAMES = [
    r"$\gamma$",
    r"$\nu_{word}$",
    r"$\nu_{picture}$",
    r"$a_{congruent}$",
    r"$a_{incongruent}$",

    r"$\tau_{correct}$",
    r"$\tau_{error}$",
    r"$s_{\nu}$",
    r"$s_{\tau}$"
]


prior = bf.simulation.Prior(prior_fun=ps.gsr_ddm_prior_fun, param_names=PARAM_NAMES)
prior_means, prior_stds = prior.estimate_means_and_stds(n_draws=10000)
prior_means = np.round(prior_means, decimals=1)
prior_stds = np.round(prior_stds, decimals=1)


simulator = bf.simulation.Simulator(simulator_fun=ps.gsr_ddm_simulator_fun)

model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator, name="GSR_DDM")

def configurator(forward_dict):
    """Configure the output of the GenerativeModel for a BayesFlow setup."""

    out_dict = {}
    out_dict["summary_conditions"] = forward_dict["sim_data"]
    params = forward_dict["prior_draws"].astype(np.float32)
    # Standardize parameters
    out_dict["parameters"] = (params - prior_means) / prior_stds
    
    return out_dict

summary_net = bf.networks.SetTransformer(input_dim=4, summary_dim=20, name="ddm_summary")

inference_net = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_settings={"dense_args": dict(kernel_regularizer=None), "dropout": False},
    coupling_design="spline",
    name="ddm_inference")

amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net, name="ddm_amortizer",
                                            summary_loss_fun='MMD')

trainer = bf.trainers.Trainer(
    generative_model=model, amortizer=amortizer, configurator= configurator,
    checkpoint_path="gsr_ddm_120_net1")

INFO:root:Performing 2 pilot runs with the GSR_DDM model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 9)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 120, 4)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.
2024-10-24 12:06:24.431476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-24 12:06:24.447959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at leas

In [4]:
#This is where the magic happens

# 1. Store all data-set chunk names in a list
datasets = sorted(os.listdir(PATH))

# 2. For each chunk
for dataset_name in datasets:
    
        # 2.1 Load chunk
        loaded_pickle = pickle.load(open(PATH +str(dataset_name), "rb" ))
        
        X_test = loaded_pickle['data_array']
        y_test = loaded_pickle['outcome_array']
        rt_summaries = loaded_pickle['rt_summaries']
        
        print(str(dataset_name)+ " loaded")


        # 2.2 Estimate chunk

        samples_dm = np.concatenate([amortizer.sample(input_dict = {"summary_conditions": x}, n_samples=3000,
                                                  to_numpy=True) for x in np.array_split(X_test, 100)], axis=0)

        samples_dm = samples_dm * prior_stds + prior_means
        
        # Discard negative samples for positively bounded parameters
        samples_dm[:,:,3:9][samples_dm[:,:,3:9] < 0] = np.nan 
        # Discard samples > 1 for gamma
        samples_dm[:,:,0][samples_dm[:,:,0] > 1] = np.nan 
        # Discard samples < -1 for gamma
        samples_dm[:,:,0][samples_dm[:,:,0] < -1] = np.nan 
        
        print(str(np.round(np.sum(np.isnan(samples_dm))/
                           (np.sum(np.isnan(samples_dm))+np.sum(~np.isnan(samples_dm))),
                           5)*100) +"% out of prior samples rejected")
        print(str(dataset_name)+ " inference done")

        # 2.3 Compute summaries of parameter posteriors: means, medians, stds, Q0.025, Q0.0975
        
        estimates = ps.compute_summaries(samples_dm)
        
        # Exclude people with less than 1000 proper prior samples for at least one parameter
        estimates[np.sum((np.sum(np.isnan(samples_dm), axis=1)>2000), axis=1)>0,:] = np.nan 
        
        print(str(np.sum((np.sum(np.isnan(samples_dm), axis=1))>2000)) +
              " people with <1000 proper samples (for at least one parameter) excluded.")

        # 2.4 Get empirical Mahalanobis distances for summary statistics provided by network
            
        summary_statistics_empirical = np.concatenate([trainer.amortizer.summary_net (x)
                                                       for x in np.array_split(X_test, 100)], axis=0)
        
        cov = EmpiricalCovariance().fit(summary_statistics_empirical)

        mahalanobis_empirical = cov.mahalanobis(summary_statistics_empirical)
        
        print(str(dataset_name)+ " Mahalanobis check done")

        
        # 2.5 Store everything together (serialized, pickle.dump) as a dict with keys 
        dict_to_store = {'data_array': X_test, 'est_array': estimates, "outcome_array": y_test,
                        'mahalanobis': mahalanobis_empirical, 'rt_summaries': rt_summaries}
        pickle.dump(dict_to_store,
                    open(PATH_TO_SAVE +"estimates_" +str(dataset_name),"wb"))
        print(str(dataset_name)+ " saving done")

# 3. Celebrate

prepared_False2003iat.p loaded
0.015% out of prior samples rejected
prepared_False2003iat.p inference done
0 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2003iat.p Mahalanobis check done
prepared_False2003iat.p saving done
prepared_False2004iat.p loaded
0.019% out of prior samples rejected
prepared_False2004iat.p inference done
3 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2004iat.p Mahalanobis check done
prepared_False2004iat.p saving done
prepared_False2004iat2.p loaded
0.021% out of prior samples rejected
prepared_False2004iat2.p inference done
1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2004iat2.p Mahalanobis check done
prepared_False2004iat2.p saving done
prepared_False2005iat.p loaded
0.02% out of prior samples rejected
prepared_False2005iat.p inference done
1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2005iat.p 

prepared_False2009iat2.p Mahalanobis check done
prepared_False2009iat2.p saving done
prepared_False2009iat3.p loaded
0.018000000000000002% out of prior samples rejected
prepared_False2009iat3.p inference done
2 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2009iat3.p Mahalanobis check done
prepared_False2009iat3.p saving done
prepared_False2009iat4.p loaded
0.019% out of prior samples rejected
prepared_False2009iat4.p inference done
1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2009iat4.p Mahalanobis check done
prepared_False2009iat4.p saving done
prepared_False2009iat5.p loaded
0.017% out of prior samples rejected
prepared_False2009iat5.p inference done
0 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2009iat5.p Mahalanobis check done
prepared_False2009iat5.p saving done
prepared_False2009iat6.p loaded
0.019% out of prior samples rejected
prepared_False2009iat6.p in

/home/mischa/Documents/bayesflow/prj_real_life_ddm/git/priors_and_simulators.py:499: RuntimeWarning: Mean of empty slice
  param_means = np.nanmean(samples_dm, axis=1)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2011iat.p Mahalanobis check done
prepared_False2011iat.p saving done
prepared_False2011iat2.p loaded
0.019% out of prior samples rejected
prepared_False2011iat2.p inference done
3 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2011iat2.p Mahalanobis check done
prepared_False2011iat2.p saving done
prepared_False2011iat3.p loaded
0.02% out of prior samples rejected
prepared_False2011iat3.p inference done
2 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2011iat3.p Mahalanobis check done
prepared_False2011iat3.p saving done
prepared_False2011iat4.p loaded
0.019% out of prior samples rejected
prepared_False2011iat4.p inference done
1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_False2011iat4.p Mahalanobis check done
prepared_False2011iat4.p saving done
prepared_False2011iat5.p loaded
0.019% o

/home/mischa/Documents/bayesflow/prj_real_life_ddm/git/priors_and_simulators.py:499: RuntimeWarning: Mean of empty slice
  param_means = np.nanmean(samples_dm, axis=1)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


...

/home/mischa/Documents/bayesflow/prj_real_life_ddm/git/priors_and_simulators.py:499: RuntimeWarning: Mean of empty slice
  param_means = np.nanmean(samples_dm, axis=1)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


4 people with <1000 proper samples (for at least one parameter) excluded.
prepared_True20222iat5.p Mahalanobis check done
prepared_True20222iat5.p saving done
prepared_True20222iat6.p loaded
0.03% out of prior samples rejected
prepared_True20222iat6.p inference done
1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_True20222iat6.p Mahalanobis check done
prepared_True20222iat6.p saving done
prepared_True20231iat.p loaded


0.031% out of prior samples rejected
prepared_True20231iat.p inference done


/home/mischa/Documents/bayesflow/prj_real_life_ddm/git/priors_and_simulators.py:499: RuntimeWarning: Mean of empty slice
  param_means = np.nanmean(samples_dm, axis=1)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


6 people with <1000 proper samples (for at least one parameter) excluded.
prepared_True20231iat.p Mahalanobis check done
prepared_True20231iat.p saving done
prepared_True20231iat2.p loaded
0.03% out of prior samples rejected
prepared_True20231iat2.p inference done
3 people with <1000 proper samples (for at least one parameter) excluded.
prepared_True20231iat2.p Mahalanobis check done
prepared_True20231iat2.p saving done
prepared_True20231iat3.p loaded
0.031% out of prior samples rejected
prepared_True20231iat3.p inference done
1 people with <1000 proper samples (for at least one parameter) excluded.
prepared_True20231iat3.p Mahalanobis check done
prepared_True20231iat3.p saving done
prepared_True20231iat4.p loaded
0.029% out of prior samples rejected
prepared_True20231iat4.p inference done
3 people with <1000 proper samples (for at least one parameter) excluded.
prepared_True20231iat4.p Mahalanobis check done
prepared_True20231iat4.p saving done
prepared_True20231iat5.p loaded
0.031% o

# Create csv 

In [5]:
df = pd.DataFrame()
datasets = os.listdir(PATH_TO_SAVE)

for dataset in datasets:
        pickles = pickle.load(open(PATH_TO_SAVE +str(dataset), "rb" ))
        df_oneset = np.concatenate((pickles['est_array'],pickles['outcome_array'],pickles['rt_summaries'],
                                   np.expand_dims(pickles['mahalanobis'], axis=1)), axis=1)
        df_onesetpd = pd.DataFrame(data = df_oneset)
        df_onesetpd["dataset"] = str(dataset)      
        df = pd.concat([df, df_onesetpd])
        
        print(str(dataset)+" done")

df.columns = ["similarity", "v_word", "v_picture", "a_congruent", "a_incongruent",
                 "tplus", "tminus", "sv", "st0",
                "similarity_median", "v_word_median", "v_picture_median", "a_congruent_median", "a_incongruent_median",
                 "tplus_median","tminus_median", "sv_median", "st0_median",
                "similarity_std", "v_wordt_std", "v_picture_std", "a_congruent_std", "a_incongruent_std",
                 "tplus_std","tminus_std", "sv_std", "st0_std",
                "similarity_q025", "v_word_q025", "v_picture_q025", "a_congruent_q025", "a_incongruent_q025",
                 "tplus_q025","tminus_q025", "sv_q025", "st0_q025",
                "similarity_q975","v_word_q975", "v_picture_q975", "a_congruent_q975", "a_incongruent_q975",
                 "tplus_q975","tminus_q975", "sv_q975", "st0_q975",
              
                  "session_id", "age",
              
                "congruent_rt_correct", "congruent_rt_correct_sd",
                "congruent_rt_correct_q25", "congruent_rt_correct_q5", "congruent_rt_correct_q75",
                "congruent_rt_error", "congruent_rt_error_sd", 
                "congruent_rt_error_q25", "congruent_rt_error_q5", "congruent_rt_error_q75",
                "congruent_accuracy",
                                              
                "incongruent_rt_correct", "incongruent_rt_correct_sd",
                "incongruent_rt_correct_q25", "incongruent_rt_correct_q5", "incongruent_rt_correct_q75",
                "incongruent_rt_error", "incongruent_rt_error_sd",
                "incongruent_rt_error_q25", "incongruent_rt_error_q5", "incongruent_rt_error_q75",
                "incongruent_accuracy",
                                              
                "word_rt_correct", "word_rt_correct_sd",
                "word_rt_correct_q25", "word_rt_correct_q5", "word_rt_correct_q75",                                             
                "word_rt_error", "word_rt_error_sd", 
                "word_rt_error_q25", "word_rt_error_q5", "word_rt_error_q75",
                "word_accuracy",
                                              
                "picture_rt_correct", "picture_rt_correct_sd",
                "picture_rt_correct_q25", "picture_rt_correct_q5", "picture_rt_correct_q75",                                             
                "picture_rt_error", "picture_rt_error_sd",
                "picture_rt_error_q25", "picture_rt_error_q5", "picture_rt_error_q75",
                "picture_accuracy",           
              
                 "mahalanobis_distance", "dataset"]

df.to_csv("df_gsr.csv", index=False)

estimates_prepared_False2015MarDeciat8.p done
estimates_prepared_False2019iat4.p done
estimates_prepared_False2019iat2.p done
estimates_prepared_True2019iat6.p done
estimates_prepared_False2009iat6.p done
estimates_prepared_False2017iat13.p done
estimates_prepared_False2021iat14.p done
estimates_prepared_False2019iat5.p done
estimates_prepared_False2012iat.p done
estimates_prepared_True2018iat8.p done
estimates_prepared_True20222iat5.p done
estimates_prepared_False2017iat9.p done
estimates_prepared_True20222iat4.p done
estimates_prepared_False2006iat.p done
estimates_prepared_False2017iat14.p done
estimates_prepared_True2020iat13.p done
estimates_prepared_True2018iat.p done
estimates_prepared_True2019iat8.p done
estimates_prepared_False2012iat3.p done
estimates_prepared_True20221iat7.p done
estimates_prepared_True2018iat7.p done
estimates_prepared_False2020iat19.p done
estimates_prepared_False2005iat3.p done
estimates_prepared_False2020iat25.p done
estimates_prepared_False2010iat4.p do

estimates_prepared_False20232iat2.p done
estimates_prepared_False20221iat5.p done
estimates_prepared_False2005iat6.p done
estimates_prepared_True2020iat22.p done
estimates_prepared_False2016JanSepiat.p done
estimates_prepared_False2020iat13.p done
estimates_prepared_False2017iat5.p done
estimates_prepared_False2008iat2.p done
estimates_prepared_False2011iat2.p done
estimates_prepared_False2020iat20.p done
estimates_prepared_False2008iat.p done
estimates_prepared_False2010iat6.p done
estimates_prepared_True2005iat4.p done
estimates_prepared_True2018iat10.p done
estimates_prepared_True2021iat7.p done
estimates_prepared_False2016JanSepiat5.p done
estimates_prepared_True2020iat21.p done
estimates_prepared_False2003iat.p done
estimates_prepared_False2005iat5.p done
estimates_prepared_False2019iat.p done
estimates_prepared_False2018iat9.p done
estimates_prepared_True2020iat17.p done
estimates_prepared_True20232iat3.p done
estimates_prepared_False2015FebMariat.p done
estimates_prepared_False2